In [86]:
import ee
import geemap
from geemap import ml
from sklearn import ensemble
import pandas as pd



# Inicializa la autenticación y la inicialización de Google Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-facuboladerasgee')

# Area de estudio

Debido al tamaño del stack de bandas es probable que el tamaño del area de estudio sea demasiado grande para exportar los datos, por lo que una alternativa es seleccionar puntos aleatorios dentro de la misma y generar un buffer sobre ellos para obetner una muestra representativa.

In [87]:
roi = ee.FeatureCollection('projects/ee-facuboladerasgee/assets/aoi_delta')
Map = geemap.Map(center=[0, 0], zoom=2)
Map.add_basemap('SATELLITE')
Map.centerObject(roi)

year = 2022

start = f'{year}-01-01'
end = f'{year}-12-31'

In [88]:
# roi_feature = ee.FeatureCollection(roi).first()
# roi_geometry = roi_feature.geometry()
# roi_coordinates = roi_geometry.coordinates().getInfo()


# roi_coords = roi_coordinates
# roi =  ee.Geometry.Polygon(roi_coords) 

### Crear Grid

In [89]:
# roi = ee.FeatureCollection('projects/ee-facuboladerasgee/assets/MalawiWoodfuelSite_1a')
# roi_geometry = roi.geometry()

# # Crear un buffer alrededor del ROI
# buffer = roi_geometry.buffer(1000)  # 10 km buffer
# buffer_clipped = buffer.difference(roi_geometry)

# # Construir la cuadrícula con celdas de 100x100 metros
# grid = roi_geometry.coveringGrid(ee.Projection('EPSG:4326'), 1000)  # 10 km grid

# # Especificar un margen de error no nulo para la operación de intersección
# error_margin = 1  # Ajusta este valor según sea necesario

# # Recortar la cuadrícula al ROI
# grid_clipped = grid.map(lambda f: f.intersection(roi_geometry, error_margin))

# # Añadir capas al mapa
# Map.addLayer(roi, {'color': 'red'}, 'ROI')
# Map.addLayer(grid_clipped, {'color': 'blue', 'fillColor': '00000000'}, 'Grid 100x100m')
# Map.centerObject(roi)
# Map

In [90]:
# # Generar 10 puntos aleatorios dentro del polígono
# random_points = ee.FeatureCollection.randomPoints(roi, 20)

# # Crear un buffer de 10 km (10000 metros) alrededor de cada punto
# buffered_points = random_points.map(lambda feature: feature.buffer(5000))

# # Función para añadir las geometrías al mapa
# def add_geometries_to_map(geometries, style, name):
#     Map.addLayer(geometries, style, name)

# # Añadir los puntos aleatorios al mapa
# add_geometries_to_map(random_points, {}, 'Random Points')

# # Añadir los buffers al mapa
# add_geometries_to_map(buffered_points, {'color': 'red'}, 'Buffered Points (10 km)')

# roi = buffered_points
# # Mostrar el mapa
# Map

### Crear buffers equidistantes

In [91]:
# def crear_buffer(feature_collection, distancia_km):
#   def crear_buffer_feature(feature):
#     buffer = feature.buffer(distancia_km * 2000)  # Convertimos km a metros
#     return buffer

#   buffers = feature_collection.map(crear_buffer_feature)

#   return buffers

# # Cargamos la FeatureCollection
# roi = ee.FeatureCollection('projects/ee-facuboladerasgee/assets/puntos_malawi')
# buffers_5km = crear_buffer(roi, 2)
# roi = buffers_5km

# # Visualizamos los buffers (opcional)
# Map.addLayer(buffers_5km, {'color': 'blue'}, 'Buffers de 5km')
# Map

# Llamado a las colecciones

### Optico

In [92]:
l7_col = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
    .filterBounds(roi) \
    .filterDate(start, end) \
    .filter(ee.Filter.lt('CLOUD_COVER', 10))

def mask_clouds_landsat(image):
    
    qa = image.select('QA_PIXEL')
    cloud_shadow_bit_mask = ee.Number(2).pow(3).int()
    clouds_bit_mask = ee.Number(2).pow(5).int()

    mask = qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0) \
        .And(qa.bitwiseAnd(clouds_bit_mask).eq(0))

    return image.updateMask(mask)

l7_masked = l7_col.map(mask_clouds_landsat)
image = l7_masked.median().toFloat().clip(roi)

ndvi = image.normalizedDifference(['SR_B4', 'SR_B3']).rename('NDVI')
mndwi = image.normalizedDifference(['SR_B2', 'SR_B5']).rename('MNDWI')
ndbi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDBI')

evi = image.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
    {
        'NIR': image.select('SR_B4'),
        'RED': image.select('SR_B3'),
        'BLUE': image.select('SR_B1')
    }
).rename('EVI')

savi = image.expression(
    '((NIR - RED) / (NIR + RED + 0.5)) * 1.5',
    {
        'NIR': image.select('SR_B4'),
        'RED': image.select('SR_B3')
    }
).rename('SAVI')

image = image.addBands([ndvi, mndwi, ndbi, evi, savi])

dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
    .filterBounds(roi) \
    .filterDate(start, end)

dem = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(roi)
slope = ee.Terrain.slope(dem).rename('slope').clip(roi)
aspect = ee.Terrain.aspect(dem).rename('aspect').clip(roi)

image = image.addBands(dem.rename('elevation'))
image = image.addBands(slope)
image = image.addBands(aspect)

In [93]:
def calculate_texture_metrics(image, index_name, roi, scale_factor=100, size=3):
    
    index_norm = image.select(index_name).multiply(scale_factor).add(scale_factor).toInt32().clip(roi)
    
    glcm = index_norm.glcmTexture(size=size)
    contrast = glcm.select(f'{index_name}_contrast').rename(f'{index_name}_Contrast')
    correlation = glcm.select(f'{index_name}_corr').rename(f'{index_name}_Correlation')
    entropy = glcm.select(f'{index_name}_ent').rename(f'{index_name}_Entropy')
    inertia = glcm.select(f'{index_name}_inertia').rename(f'{index_name}_Inertia')
    
    # Agregar las bandas de textura a la imagen original
    return image.addBands([contrast, correlation, entropy, inertia])

def calculate_texture_stats(image, index_name, roi, scale_factor=10, size=3):
    
    index_norm = image.select(index_name).multiply(scale_factor).add(scale_factor).toInt32().clip(roi)
    glcm = index_norm.glcmTexture(size=size)
    
    texture_metrics = [
        f'{index_name}_asm', f'{index_name}_contrast', f'{index_name}_corr', f'{index_name}_var',
        f'{index_name}_idm', f'{index_name}_savg', f'{index_name}_svar', f'{index_name}_sent',
        f'{index_name}_ent', f'{index_name}_dvar', f'{index_name}_dent', f'{index_name}_imcorr1',
        f'{index_name}_imcorr2', f'{index_name}_maxcorr', f'{index_name}_diss', f'{index_name}_inertia',
        f'{index_name}_shade', f'{index_name}_prom'
    ]
    
    texture_bands = glcm.select(texture_metrics)
    
    mean_texture = texture_bands.reduce(ee.Reducer.mean()).rename(f'{index_name}_Mean_Texture')
    variance_texture = texture_bands.reduce(ee.Reducer.variance()).rename(f'{index_name}_Variance_Texture')   

    image = image.addBands([mean_texture, variance_texture])
    
    return image


image = calculate_texture_metrics(image, 'NDVI', roi)
image = calculate_texture_metrics(image, 'SAVI', roi)
image = calculate_texture_stats(image, 'NDVI', roi)
image = calculate_texture_stats(image, 'SAVI', roi)

In [94]:
Map.centerObject(roi, 10)
Map.addLayer(image.select('NDVI_Contrast'), {}, 'image', False)
Map

Map(center=[-33.686057360582126, -58.66210629857657], controls=(WidgetControl(options=['position', 'transparen…

# Palsar


In [95]:
import math

def terrain_correction_palsar(image, roi, angle_image):
    # Usar la ROI para recortar el SRTM
    srtm = ee.Image('USGS/SRTMGL1_003').clip(roi)
    
    # Convertir la imagen a dB usando la fórmula γ₀ = 10 log₁₀(DN²) - 83.0 dB

    dn_squared = image.pow(2)   
    backscatter = ee.Image.constant(10).multiply(dn_squared.log10()).subtract(83.0)
    theta_i = angle_image  
    
    # Obtener el aspecto promedio de la región de interés
    phi_i = ee.Terrain.aspect(theta_i).reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=roi,  # Usar la ROI
        scale=1000
    ).get('aspect')
    
    # Convertir phi_i a una imagen constante
    phi_i_image = ee.Image.constant(phi_i)
    
    # Geometría del terreno
    alpha_s = ee.Terrain.slope(srtm).select('slope')
    phi_s = ee.Terrain.aspect(srtm).select('aspect')
    
    # Geometría del modelo
    phi_r = phi_i_image.subtract(phi_s)
    
    # Convertir todo a radianes
    phi_r_rad = phi_r.multiply(math.pi / 180)
    alpha_s_rad = alpha_s.multiply(math.pi / 180)
    theta_i_rad = theta_i.multiply(math.pi / 180)
    ninety_rad = ee.Image.constant(90).multiply(math.pi / 180)

    # Pendiente en el rango
    alpha_r = (alpha_s_rad.tan().multiply(phi_r_rad.cos())).atan()

    # Pendiente en acimut
    alpha_az = (alpha_s_rad.tan().multiply(phi_r_rad.sin())).atan()

    # Ángulo de incidencia local
    theta_lia = (alpha_az.cos().multiply((theta_i_rad.subtract(alpha_r)).cos())).acos()
    theta_lia_deg = theta_lia.multiply(180 / math.pi)

    # Gamma_nought_flat usando la fórmula ajustada
    gamma0 = backscatter.divide(theta_i_rad.cos())
    gamma0_dB = gamma0.rename('gamma0_dB')

    # Modelo volumétrico
    nominator = (ninety_rad.subtract(theta_i_rad).add(alpha_r)).tan()
    denominator = (ninety_rad.subtract(theta_i_rad)).tan()
    vol_model = (nominator.divide(denominator)).abs()

    # Aplicar el modelo
    gamma0_volume = gamma0.divide(vol_model)
    gamma0_volume_dB = gamma0_volume.rename('gamma0_volume_dB')

    return gamma0_dB


def preprocess(image, roi, angle_image):
    image = terrain_correction_palsar(image, roi, angle_image)
    speckle_filtered = image.focal_median(kernelType='circle', radius=50, units='meters')
    return speckle_filtered


angle = (
    ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR_EPOCH')
    .filterBounds(roi)
    .filterDate(start, end)
    .select('angle')  
    .median() 
)

img_hh = (
    ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR_EPOCH')
    .filterBounds(roi)
    .filterDate(start, end)
    .select('HH')  # Seleccionar solo HH
    .map(lambda img: preprocess(img, roi, angle))  # Pasar roi y angle_image al preprocesamiento
)

img_hv = (
    ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR_EPOCH')
    .filterBounds(roi)
    .filterDate(start, end)
    .select('HV')  # Seleccionar solo HV
    .map(lambda img: preprocess(img, roi, angle))  # Pasar roi y angle_image al preprocesamiento
)

palsar_hh_median = img_hh.median().rename('HH_dB')
palsar_hv_median = img_hv.median().rename('HV_dB')

hh_plus_hv = palsar_hh_median.add(palsar_hv_median).rename('HH_plus_HV')
hh_minus_hv = palsar_hh_median.subtract(palsar_hv_median).rename('HH_minus_HV')
hh_div_hv = palsar_hh_median.divide(palsar_hv_median).rename('HH_div_HV')
rvi = palsar_hv_median.divide(palsar_hh_median.add(palsar_hv_median)).multiply(4).rename('RVI_palsar')

palsar_combined = ee.Image.cat([palsar_hh_median, palsar_hv_median, hh_plus_hv, hh_minus_hv, hh_div_hv, rvi]).clip(roi)
image = image.addBands(palsar_combined)

In [96]:
def calculate_texture_metrics(image, band_name, roi, size=5):
    # Convertir la banda a tipo int32 (no se realiza normalización adicional)
    band_int = image.select(band_name).toInt32().clip(roi)
    
    # Calcular texturas usando la banda convertida a int32
    glcm = band_int.glcmTexture(size=size)
    
    # Extraer las texturas de interés y renombrarlas
    contrast = glcm.select(f'{band_name}_contrast').rename(f'{band_name}_Contrast')
    correlation = glcm.select(f'{band_name}_corr').rename(f'{band_name}_Correlation')
    entropy = glcm.select(f'{band_name}_ent').rename(f'{band_name}_Entropy')
    inertia = glcm.select(f'{band_name}_inertia').rename(f'{band_name}_Inertia')
    
    # Agregar las bandas de textura a la imagen original
    return image.addBands([contrast, correlation, entropy, inertia])



# def calculate_texture_stats(image, index_name, roi, size=5):
#     # Normalizar el índice y convertirlo a tipo int32
#     index_norm = image.select(index_name).divide(1000).toInt32().clip(roi)  # Normalización simple
#     glcm = index_norm.glcmTexture(size=size)
    
#     # Lista de todas las métricas de textura que queremos calcular
#     texture_metrics = [
#         f'{index_name}_asm', f'{index_name}_contrast', f'{index_name}_corr', f'{index_name}_var',
#         f'{index_name}_idm', f'{index_name}_savg', f'{index_name}_svar', f'{index_name}_sent',
#         f'{index_name}_ent', f'{index_name}_dvar', f'{index_name}_dent', f'{index_name}_imcorr1',
#         f'{index_name}_imcorr2', f'{index_name}_maxcorr', f'{index_name}_diss', f'{index_name}_inertia',
#         f'{index_name}_shade', f'{index_name}_prom'
#     ]
    
#     # Extraer las texturas de interés
#     texture_bands = glcm.select(texture_metrics)
    
#     # Calcular la media y varianza de las bandas de textura
#     mean_texture = texture_bands.reduce(ee.Reducer.mean()).rename(f'{index_name}_Mean_Texture')
#     variance_texture = texture_bands.reduce(ee.Reducer.variance()).rename(f'{index_name}_Variance_Texture')    

#     # Agregar las bandas de textura y las métricas adicionales a la imagen original
#     image = image.addBands([mean_texture, variance_texture])
    
#     return image


image = calculate_texture_metrics(image, 'HH_dB', roi)
# image = calculate_texture_stats(image, 'HH_dB',roi)
image = calculate_texture_metrics(image, 'HV_dB', roi)
# image = calculate_texture_stats(image, 'HV_dB',roi)

In [97]:
bands = [
    'SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 
    'MNDWI', 'NDBI', 'NDVI', 'EVI', 'SAVI', 
    'elevation', 'slope', 'aspect',
    'HH_dB', 'HV_dB', 
    'HH_plus_HV', 'HH_minus_HV', 'HH_div_HV', 'RVI_palsar',
    
    # Métricas de textura para NDVI
    'NDVI_Contrast', 'NDVI_Correlation', 'NDVI_Entropy', 'NDVI_Inertia',
    'NDVI_Mean_Texture', 'NDVI_Variance_Texture',
    
    # Métricas de textura para SAVI
    'SAVI_Contrast', 'SAVI_Correlation', 'SAVI_Entropy', 'SAVI_Inertia',
    'SAVI_Mean_Texture', 'SAVI_Variance_Texture',
    
    # Métricas de textura para HH_dB
    'HH_dB_Contrast', 'HH_dB_Correlation', 'HH_dB_Entropy', 'HH_dB_Inertia',
    
    
    # Métricas de textura para HV_dB
    'HV_dB_Contrast', 'HV_dB_Correlation', 'HV_dB_Entropy', 'HV_dB_Inertia',
    
]


image = image.select(bands)

In [98]:
Map.centerObject(roi, 10)
Map.addLayer(image.select('HH_dB_Contrast'), {}, 'image', False)
Map

Map(center=[-33.686057360582126, -58.66210629857657], controls=(WidgetControl(options=['position', 'transparen…

# Muestreo de datos

En esta seccion se genera la logica de llamado a los datos de GEDI y recoleccion de datos sobre sus pixeles, la exportacion del csv se gerena por partes para cada uno de los buffers de puntos creados anteriormente.  

In [99]:
# Función para filtrar y seleccionar las bandas relevantes de GEDI
def get_gedi_data(band_name):
    return ee.ImageCollection('LARSE/GEDI/GEDI04_A_002_MONTHLY') \
        .filterBounds(roi) \
        .filterDate(start, end) \
        .map(lambda image: image.updateMask(
            image.select('degrade_flag').eq(0)
            .And(image.select('l2_quality_flag'))
            .And(image.select('l4_quality_flag')))) \
        .select(band_name) \
        .median() \
        .toFloat() \
        .clip(roi)

# Obtener las bandas agbd, agbd_se, y l4_quality_flag
gediData_agbd = get_gedi_data('agbd').rename('agbd')
gediData_agbd_t_se = get_gedi_data('agbd_se').rename('agbd_se')

# Combinar todas las bandas en una sola imagen
gedi_combined = ee.Image.cat([gediData_agbd, gediData_agbd_t_se])

# Añadir las bandas combinadas a la imagen original
image = image.addBands(gedi_combined)

# Convertir la imagen a float si es necesario
def convert_to_float(image):
    return image.float()

# Convertir todas las bandas de la imagen a float
image = convert_to_float(image)

In [100]:
import time

sample = image.addBands(gediData_agbd).updateMask(gediData_agbd).sample(
        scale=30,
        region=roi,
        geometries=True
    )

export_task = ee.batch.Export.table.toDrive(
        collection=sample,
        description='ExportSampleToCSV',
        folder='EE_delta',
        fileNamePrefix=f'Datos_RF_2022',
        fileFormat='CSV'
    )

# Iniciar la tarea de exportación
export_task.start()

    # Esperar a que la tarea de exportación se complete
export_task.status()

    # Verificar el estado de la tarea y mostrar un mensaje de éxito
while export_task.active():
    print('Exportación en progreso...')
    time.sleep(30)  # Esperar 30 segundos antes de verificar el estado nuevamente
    
    if export_task.status()['state'] == 'COMPLETED':
        print(f'Exportación completada con éxito.')
    else:
        print(f'Error en la exportación: {export_task.status()}')

Exportación en progreso...
Error en la exportación: {'state': 'RUNNING', 'description': 'ExportSampleToCSV', 'priority': 100, 'creation_timestamp_ms': 1724782217229, 'update_timestamp_ms': 1724782240862, 'start_timestamp_ms': 1724782220702, 'task_type': 'EXPORT_FEATURES', 'attempt': 1, 'id': 'OAHG6RBNNT33KA2RMBRSNER3', 'name': 'projects/ee-facuboladerasgee/operations/OAHG6RBNNT33KA2RMBRSNER3'}
Exportación en progreso...
Exportación completada con éxito.


In [101]:
# import time

# # Función para exportar cada buffer individualmente
# def export_buffer(buffer, index):
#     sample = image.addBands(gediData_agbd).updateMask(gediData_agbd).sample(
#         scale=30,
#         region=buffer.geometry(),
#         geometries=True
#     )

#     export_task = ee.batch.Export.table.toDrive(
#         collection=sample,
#         description=f'ExportSampleToCSV_{index}',
#         folder='EE_malawi2019-Landast-Palsar',
#         fileNamePrefix=f'Datos_RF_{index}',
#         fileFormat='CSV'
#     )

#     # Iniciar la tarea de exportación
#     export_task.start()

#     # Esperar a que la tarea de exportación se complete
#     export_task.status()

#     # Verificar el estado de la tarea y mostrar un mensaje de éxito
#     while export_task.active():
#         print('Exportación en progreso...')
#         time.sleep(30)  # Esperar 30 segundos antes de verificar el estado nuevamente

#     if export_task.status()['state'] == 'COMPLETED':
#         print(f'Exportación {index} completada con éxito.')
#     else:
#         print(f'Error en la exportación {index}: {export_task.status()}')

# # Ejemplo de uso
# buffered_points_list = buffers_5km.toList(buffers_5km.size())

# for i in range(buffers_5km.size().getInfo()):
#     buffer = ee.Feature(buffered_points_list.get(i))
#     export_buffer(buffer, i)